# Pipeline examples

This example show quickly how to use pipelines in `redis-py`.

## Checking that Redis is running

In [1]:
import redis 

r = redis.Redis(decode_responses=True)
r.ping()

True

## Simple example

### Creating a pipeline instance

In [2]:
pipe = r.pipeline()

### Adding commands to the pipeline

In [3]:
pipe.set("a", "a value")
pipe.set("b", "b value")

pipe.get("a")

Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

### Executing the pipeline

In [4]:
pipe.execute()

[True, True, 'a value']

The responses of the three commands are stored in a list. In the above example, the two first boolean indicates that the the `set` commands were successfull and the last element of the list is the result of the `get("a")` comand.

## Chained call

The same result as above can be obtained in one line of code by chaining the opperations.

In [5]:
pipe = r.pipeline()
pipe.set("a", "a value").set("b", "b value").get("a").execute()

[True, True, 'a value']

## Performance comparison

Using pipelines can improve performance, for more informations, see [Redis documentation about pipelining](https://redis.io/docs/manual/pipelining/). Here is a simple comparison test of performance between basic and pipelined commands (we simply increment a value and measure the time taken by both method).

In [6]:
from datetime import datetime

incr_value = 100000

### Without pipeline

In [7]:
r.set("incr_key", "0")

start = datetime.now()

for _ in range(incr_value):
    r.incr("incr_key")
res_without_pipeline = r.get("incr_key")

time_without_pipeline = (datetime.now() - start).total_seconds()

In [8]:
print("Without pipeline")
print("================")
print("Time taken: ", time_without_pipeline)
print("Increment value: ", res_without_pipeline)

Without pipeline
Time taken:  21.759733
Increment value:  100000


### With pipeline

In [9]:
r.set("incr_key", "0")

start = datetime.now()

pipe = r.pipeline()
for _ in range(incr_value):
    pipe.incr("incr_key")
pipe.get("incr_key")
res_with_pipeline = pipe.execute()[-1]

time_with_pipeline = (datetime.now() - start).total_seconds()

In [10]:
print("With pipeline")
print("=============")
print("Time taken: ", time_with_pipeline)
print("Increment value: ", res_with_pipeline)

With pipeline
Time taken:  2.357863
Increment value:  100000


Using pipelines provides the same result in much less time.